# 1. Install and Import Dependencies

In [5]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [8]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\r1005\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\r1005\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# 3. Encode and Calculate Sentiment

In [9]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [10]:
result = model(tokens)

In [11]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [12]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [19]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [21]:
r.text


'<html lang="en"><head><title>yelp.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMAxtQYbSa5m6IAUgkP6Q==\',\'hsh\':\'3BD2468BAE4D73BEA0B5DE8314D745\',\'t\':\'bv\',\'qp\':\'\',\'s\':50888,\'e\':\'636d989ec4b5d5957b0708ff6f0d4b80ebd396cb8b06b5ae09b263275a388348\',\'host\':\'geo.captcha-delivery.com\',\'cookie\':\'rBsZ08rfce4o8ExJEcpCRSu8dAROQCoCXYDtoklY5vXM6gWLzuQK9vhqsrJw3gd3IjeVtmB61VBxM2S96J88SpXFqBSvuZPQns3kiZEciOcHtUO4mYJix_XDDL3CNv8B\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>'

In [20]:
reviews

[]

# 5. Load Reviews into DataFrame and Score

In [15]:
import numpy as np
import pandas as pd

In [16]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [17]:
df['review'].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

In [ ]:
df['review'].iloc[3]